In [81]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark import SQLContext
import random

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [82]:
lista_est = []
for i in range(100):
    lista_est.append((i//5, i%5, random.randint(0,180)))

lista_part = []
for i in range(20):
    lista_part.append((i, 2019+(i//10), ((i//5)%2)+1))

rdd_est = sc.parallelize(lista_est)
rdd_part = sc.parallelize(lista_part)

In [83]:
# rdd_est = (id_partido, id_jugador, tiempo)
# rdd_part = (id_partido, año, semestre)
rdd_est = rdd_est.map(lambda x: (x[0], (x[1], x[2]))) 
rdd_part = rdd_part.map(lambda x: (x[0], (x[1], x[2])))

In [84]:
rdd_ext = rdd_est.join(rdd_part) # (id_partido, ((id_jugador, tiempo), (año, semestre)))
rdd_ext.take(5)

[(0, ((0, 157), (2019, 1))),
 (0, ((1, 40), (2019, 1))),
 (0, ((2, 180), (2019, 1))),
 (0, ((3, 102), (2019, 1))),
 (0, ((4, 79), (2019, 1)))]

In [85]:
rdd_ext = rdd_ext.map(lambda x: ((x[1][0][0], x[1][1][0], x[1][1][1]), x[1][0][1])).reduceByKey(lambda a,b: a+b) # ((id_jugador, año, semestre), total_jugado)
rdd_ext.take(5)

[((1, 2019, 1), 300),
 ((4, 2020, 1), 376),
 ((2, 2019, 1), 563),
 ((0, 2020, 2), 465),
 ((3, 2020, 1), 377)]

In [86]:
rdd_ext = rdd_ext.map(lambda x: (x[0][0], (str(x[0][1])+'-'+str(x[0][2]), x[1])))# (id_jugador, ('YYYY-SS', tiempo_jugado))
rdd_ext.take(5)

[(1, ('2019-1', 300)),
 (4, ('2020-1', 376)),
 (2, ('2019-1', 563)),
 (0, ('2020-2', 465)),
 (3, ('2020-1', 377))]

In [87]:
rdd_ext = rdd_ext.groupByKey().map(lambda x: (x[0], sorted(list(x[1]))))
rdd_ext.take(5)

[(0, [('2019-1', 441), ('2019-2', 375), ('2020-1', 141), ('2020-2', 465)]),
 (1, [('2019-1', 300), ('2019-2', 700), ('2020-1', 346), ('2020-2', 509)]),
 (2, [('2019-1', 563), ('2019-2', 393), ('2020-1', 561), ('2020-2', 505)]),
 (3, [('2019-1', 400), ('2019-2', 452), ('2020-1', 377), ('2020-2', 503)]),
 (4, [('2019-1', 394), ('2019-2', 465), ('2020-1', 376), ('2020-2', 542)])]

In [105]:
rdd_aumento = rdd_ext.map(lambda x: (x[0], [(x[1][i][1]-x[1][i-1][1])*100/x[1][i-1][1] for i in range(1,len(x[1]))]))
rdd_aumento.take(5)

[(0, [-14.965986394557824, -62.4, 229.7872340425532]),
 (1, [133.33333333333334, -50.57142857142857, 47.10982658959538]),
 (2, [-30.19538188277087, 42.74809160305343, -9.982174688057041]),
 (3, [13.0, -16.5929203539823, 33.42175066312998]),
 (4, [18.02030456852792, -19.13978494623656, 44.148936170212764])]

In [113]:
rdd_aumento.filter(lambda x: any(num > 50 for num in x[1])).take(5)

[(0, [-14.965986394557824, -62.4, 229.7872340425532]),
 (1, [133.33333333333334, -50.57142857142857, 47.10982658959538])]

True